# Adding data

## Create a project

This can be done only using the web interface

They cannot be deleted by user. Write to Mike Gavrielides to get help

## Add mice

- Can work only from a computer with a web browser and a graphical interface.
- Need to be on the crick network (vpn)
- Requires the `download_folder` in the config file.

Example:

In [ ]:
!flexiznam add-mouse -p demo_project -m PZAJ5.1a

## Add data using flexilims web interface

### Upload CSV

A bit cumbersome. I don't use it but it works. You can try it.

### As a child of an existing entity

Just go to the entity page and click "Add" in the right margin.

- Easy to do
- Slow
- Cannot set custom attributes

## Using Flexilims

The low level API can be access directly

In [ ]:
from flexilims import Flexilims
import flexiznam as flm

# create a session
user = flm.PARAMETERS["flexilims_username"]
flm_sess = Flexilims(
    username=user, password=flm.get_password(username=user, app="flexilims")
)

flm_sess.post(
    datatype="dataset",
    name="important_metadata",
    origin_id="6109ac55a651ff0b6237e101",
    project_id="610989f9a651ff0b6237e0f6",
    attributes=dict(
        path="PZAJ5.1a/metadata.txt", dataset_type="other", custom_field="very personal"
    ),
    strict_validation=False,
)

## Using flexiznam

The low level API is pain. ID in particular are annoying. Flexiznam has some functions to simplify that and make sure you add element that will be understood by the rest of the codebase

In [ ]:
import flexiznam as flm

flm_sess = flm.get_flexilims_session(project_id="demo_project")

Check:

```
flm.add_entity
flm.add_dataset
flm.add_experimental_session
flm.add_recording
flm.add_sample
```

In [ ]:
# get parent id
flm.add_dataset(
    parent_id="6109ac55a651ff0b6237e101",
    dataset_type="microscopy",
    created="2021-08-04 10:58:29",
    path="PZAJ5.1a/image.png",
    is_raw="no",
    flexilims_session=flm_sess,
    dataset_name="pretty_picture",
    strict_validation=False,
)

## Using yaml files

### Physiology data

Data is organised in `Session` and `Recording`. Both can contain `Datasets`. The minimum info for `flexiznam` to find the data can be given in a yaml file like that:

In [ ]:
%%writefile /Volumes/lab-znamenskiyp/home/shared/projects/demo_project/physio_acq.yml

project: "demo_project"
mouse: "PZAJ5.1a"
session: "S20210513"
recordings:
  R182758_SphereCylinder:
    protocol: "SphereCylinder"
    notes: "This is the best recording"
  R193432_Retinotopy:
    protocol: "Retinotopy"
    attributes:
        objective: "40x"
        list_of_stuff: [1, 2, "three"]

Then you can use `flexiznam process-yaml` to make sure that everything is fine

```
cd /Volumes/lab-znamenskiyp/home/shared/projects/demo_project/
flexiznam process-yaml --source_yaml physio_acq.yml
```

Check the generated file and look for `XXERRORXX` flag.
Then you can upload to `flexilims`

```
flexiznam yaml-to-flexilims --source_yaml physio_acq_autogenerated_full_file.yml
```

### Other data

Data that is not organised in sessions, can be attached to `sample` instead. The rest is similar

In [ ]:
%%writefile /Volumes/lab-znamenskiyp/home/shared/projects/demo_project/ex_vivo_acq.yml

project: "demo_project"
mouse: "PZAJ5.1a"
session: 
samples:
    brain:
        notes: 'perfused brain'
        samples:
            slide4:
                samples:
                    slice3:

Process the yaml

```flexiznam process-yaml --source_yaml ex_vivo_acq.yml```

And upload it:

```flexiznam yaml-to-flexilims --source_yaml ex_vivo_acq_autogenerated_full_file.yml```
